In [1]:
import requests
import pandas as pd
import json

In [2]:
#input constants
iss_date = '2023-05-30'
file_name_prev = 'imoex_tqbr_price_2022.csv'
file_name = 'imoex_tqbr_price_2023.csv'
iss_url_raw = 'https://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities.json'

In [3]:
#upload cvs data
df_prev = pd.read_csv(file_name_prev)
df_tek = pd.read_csv(file_name)
price_history = pd.concat([df_tek, df_prev])
new_day_start_index = price_history['Index'].max() + 1
price_history.set_index('Index', inplace=True)
price_history.sort_values('Date', ascending=False, inplace=True)

In [4]:
#upload lot size data
with open('lotsize.json', "r") as read_file:
    lotsize = json.load(read_file)
    read_file.close()

In [5]:
#upload day results from MOEX
tickers_list = []
iss_start = 0
tickers_count = 1
while iss_start < tickers_count:
    iss_url = iss_url_raw + '?date=' + iss_date + '&start=' + str(iss_start)
    response = requests.get(iss_url)
    raw_data = json.loads(response.text)
    if iss_start == 0:
        tickers_count = raw_data["history.cursor"]["data"][0][1]
        columns_raw = raw_data["history"]["columns"]
        column_tradedate = columns_raw.index('TRADEDATE')
        column_secid = columns_raw.index('SECID')
        column_open = columns_raw.index('OPEN')
        column_low = columns_raw.index('LOW')
        column_high = columns_raw.index('HIGH')
        column_close = columns_raw.index('CLOSE')
        column_volume = columns_raw.index('VOLUME')
    raw_data = raw_data["history"]["data"]  # list of ticker lists
    for ticker in raw_data:
        if ticker[column_volume] > 0:
            one_ticker = [ticker[column_tradedate], ticker[column_secid], ticker[column_volume], ticker[column_open], ticker[column_low], ticker[column_high], ticker[column_close], 0, 0, 0, 0, 0, 0, 0]
            tickers_list.append(one_ticker)
    iss_start += 100

In [6]:
#create and format frame for new day
new_day = pd.DataFrame(tickers_list) 
new_day.columns = ['Date', 'Ticker', 'Volume', 'Open', 'Low', 'High', 'Close', 'VolEMA10', 'EMA10', 'EMA20', 'ATR', 'TR', 'Lot_trade', 'Rank']
new_day.index += new_day_start_index
convert_dict = {'VolEMA10': float,'EMA10': float,'EMA20': float,'ATR': float,'TR': float}
new_day = new_day.astype(convert_dict)

In [7]:
#calculating additional data
no_data = []
for i, indx in enumerate(new_day.index):
    share = new_day.at[indx, 'Ticker']
    if share == 'ENRU':
        share = 'ELFV'
        new_day.at[indx, 'Ticker'] = 'ELFV'
    prev = price_history.loc[price_history.Ticker == share]
    if len(prev) > 1:
        prev = prev.index[0]
        
        tmp = new_day.at[indx,'Volume'] * 2/11 + price_history.at[prev,'VolEMA10']*(1-2/11)
        new_day.at[indx,'VolEMA10'] = round(tmp, 0)
        
        tmp = new_day.at[indx,'Close'] * 2/11 + price_history.at[prev,'EMA10']*(1-2/11)
        new_day.at[indx,'EMA10'] = round(tmp, 6)
        
        tmp = new_day.at[indx,'Close'] * 2/21 + price_history.at[prev,'EMA20']*(1-2/21)
        new_day.at[indx,'EMA20'] = round(tmp, 6)        
        
        tr1 = new_day.at[indx,'High'] - new_day.at[indx,'Low']
        tr2 = abs(new_day.at[indx,'High'] - price_history.at[prev,'Close'])
        tr3 = abs(price_history.at[prev,'Close'] - new_day.at[indx,'Low'])                    
        tmp = max(tr1, tr2, tr3)
        new_day.at[indx,'TR'] = round(tmp, 6)
        
        tmp = (price_history.at[prev,'ATR']*13 + new_day.at[indx,'TR'])/14
        new_day.at[indx,'ATR'] = round(tmp, 6)
        
        tmp = new_day.at[indx, 'VolEMA10'] / lotsize.get(share)
        new_day.at[indx,'Lot_trade'] = round(tmp, 0)
    else:
        no_data.append(indx)
print(no_data)

[]


In [8]:
#prevent SettingWithCopyWarning message from appearing and calculate rank
pd.options.mode.chained_assignment = None
new_day.sort_values('Lot_trade', ascending=True, inplace=True)
new_day['Rank']= pd.qcut(new_day['Lot_trade'], q = 10, labels = False)
for i,indx in enumerate(new_day.index):    
    new_day.at[indx, 'Rank'] = 10-new_day.at[indx, 'Rank']

In [9]:
#update csv data
new_day.to_csv(file_name, mode='a', header=False)